# Problem set 1

The performance of a client-server system is strongly influenced by two major network characteristics: the bandwidth of the network (that is, how many bits it can transport in unit time) and the latency (that is, how many seconds it takes for the first bit of the message to get from the client to the server). 

In this problemset the latter feature, the latency, is practically investigated in the Internet network.

Most entities in the Internet, call them nodes, that take part in the communication process obey the laws the IP protocols dictate and implement a layered software structure. Our investigation is in the network layer. Note the fact the messages to be sent are broken down to smaller segments, named IP packets, which are dressed up with additional necessary information to be routed to the right destination(s). The notion of IP address identifies a node in the network layer uniquely. Any nodes inside the network that take part in a given client-server communication process needs to decide how to forward IP packages accross them so the packets reach their destination. The rule of the thumb is to try be as efficient as possible.


## Problem 1.1

An IP packet travelling between the client and the server will suffer from some latency. This delay is depending on many terms. Some of these terms vary in time some are constant. The terms include the ensamblement or processing of the packets, buffering delay when the forwarder node or its ports are loaded, and the propagation delay in the mediums between any adjacent nodes.

The `ping` program allows you to send a test packet (ICMP request) to a server with a given location and see how long it takes to get there and back. 

Try and collect a set of servers for which the geographical location can be well approximated. It is best to use universities or govenmental portals since the location of their servers is known very accurately. Then using the `ping` measure how long it takes to get from your location to these known locations. From the data, plot the one-way transit time over the Internet as a function of distance. 

Draw conclusions from your plot. If necessary argue how you preprocess or filter your dataset.

If you have an access and permissions to terminal computers located in a remote sites carry out the measurements to the same set of nodes and try compare your results.

## Problem 1.2

Select few destination nodes used in P1.1 and measure the roud trip delay for long time (few weeks long). Plot and analyze the data try and find any remarks in the time evolution of the data that can be caused by human factor.

In [1]:
# Most American universities have their European servers based near Lucerne, Switzerland

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('servers.csv')
df

,IP Address,domain,Country,Region/City,lat,lon,ISP/Organization
0,148.6.2.104,wigner.hu,Hungary,Budapest,47.4980,19.0399,MTA Wigner Fizikai Kutatokozpont
1,157.181.1.232,elte.hu,Hungary,Budapest,47.4839,19.0597,Eotvos Lorand Universityof Sciences
2,193.231.20.15,ubbcluj.ro,Romania,Cluj,46.7667,23.6000,Romanian Education Network
3,193.226.5.7,utcluj.ro,Romania,Cluj,46.7727,23.5864,Romanian Education Network
4,129.187.255.151,tum.de,Germany,Garchingbei München,48.2490,11.6510,Leibniz Rechenzentrum
5,160.45.170.10,fu-berlin.de,Germany,Berlin,52.5244,13.4105,Freie Universitaet Berlin
6,130.75.2.151,uni-hannover.de,Germany,Hannover,52.3705,9.7332,Gottfried Wilhelm Leibniz Universitaet Hannover
7,131.152.228.33,unibas.ch,Switzerland,Basel,47.5584,7.5733,Universitaet Basel
8,193.49.39.51,universite-lyon.fr,France,Lyon,45.7485,4.8467,PresUniversitede Lyon
9,194.117.3.27,www.ulisboa.pt,Portugal,Lisboa,38.7167,-9.1333,Fundacao Paraa Cienciaea Tecnologia I.P.


In [4]:
import json
import subprocess
from subprocess import check_output

# Get my ip and my location
resp = check_output(["curl", "https://ipinfo.io"])
result = json.loads(resp.decode('utf8'))
my_ip = result['ip']
my_loc_lat, my_loc_lon = result['loc'].split(',')
my_loc_lat = float(my_loc_lat)
my_loc_lon = float(my_loc_lon)

In [5]:
import folium

mymap = folium.Map(
  location = [my_loc_lat, my_loc_lon],
  zoom_start = 2,
  tiles = 'cartodbpositron'
)

folium.Marker([my_loc_lat, my_loc_lon], popup='My location').add_to(mymap)

for index, row in df.iterrows():
    folium.Marker(
      [row['lat'], row['lon']],
      popup=row['domain']
    ).add_to(mymap)

mymap

In [11]:
# Run all processes asynchronously
import os
import signal
import subprocess
from subprocess import Popen, PIPE

running = []

for k in range(len(df)):
    filename = "pings-" + df[['domain']].values[k][0] + ".pingdata"
    cmd = "if [ -f " + filename + " ]; then\n"
    cmd += "rm -v " + filename + "\n"
    cmd += "fi"
    cmd = ''.join(cmd)
    subprocess.call(cmd, shell=True)

    cmd = 'printf "#My ip: {}\\n#My lat: {}\\n#My lon: {}\\n"'.format(my_ip, my_loc_lat, my_loc_lon)
    cmd += " > " + filename
    subprocess.call(cmd, shell=True)
    
    tsec=60
    print("Pinging " + str(df[['domain']].values[k][0]) + " for %d seconds..."%tsec)
    
    # Ping for a specific time interval
    # Use -t on Mac but -w on Linux
    cmd = ["ping", df[['IP Address']].values[k][0], "-w%d"%tsec, ">>", filename]
    cmd = ' '.join(cmd)
    
    # https://stackoverflow.com/a/636601/1708155
    running.append(
        {
            'command': cmd,
            'process': Popen(cmd, shell=True)
        }
    )
    
import time
while running:
    for idx, record in enumerate(running):
        retcode = record['process'].poll()
        if retcode is not None: # Process finished.
            running.remove(record)
            print("Command " + record['command'] + " finished.")
            break
        else: # No process is done, wait a bit and check again.
            time.sleep(.1)
            continue

    if retcode != 0 and idx < len(running):
        print("Command " + running[idx]['command'] + " failed.")
        running.pop(idx)

Pinging wigner.hu for 60 seconds...
Pinging elte.hu for 60 seconds...
Pinging ubbcluj.ro for 60 seconds...
Pinging utcluj.ro for 60 seconds...
Pinging tum.de for 60 seconds...
Pinging fu-berlin.de for 60 seconds...
Pinging uni-hannover.de for 60 seconds...
Pinging unibas.ch for 60 seconds...
Pinging universite-lyon.fr for 60 seconds...
Pinging www.ulisboa.pt for 60 seconds...
Pinging cam.ac.uk for 60 seconds...
Pinging ed.ac.uk for 60 seconds...
Pinging www.princeton.edu for 60 seconds...
Pinging www.harvard.edu for 60 seconds...
Pinging www.ucla.edu for 60 seconds...
Pinging www.ynu.ac.jp for 60 seconds...
Pinging www.useoul.edu for 60 seconds...
Pinging web.iit.edu for 60 seconds...
Pinging fearp.usp.br for 60 seconds...
Pinging uchile.cl for 60 seconds...
Pinging unimelb.edu.au for 60 seconds...
Command ping 43.245.43.59 -w60 >> pings-unimelb.edu.au.pingdata failed.
Command ping 200.89.76.16 -w60 >> pings-uchile.cl.pingdata failed.
Command ping 143.107.158.19 -w60 >> pings-fearp.usp